# Amazon Bedrock AgentCore コード インタープリターでコマンドを実行する - チュートリアル

このチュートリアルでは、Amazon Bedrock AgentCore コード インタープリターを使用してコマンド (シェルと AWS CLI) を実行する方法を示します。AWS サービスと対話し、特に S3 操作に焦点を当てます。以下の手順を説明します。

1. コード インタープリターの作成
2. コード インタープリター セッションの開始
3. コマンドの実行 (シェルと AWS CLI)
5. S3 操作の実行 (バケットの作成、オブジェクトのコピー、バケット オブジェクトの一覧表示)
6. クリーンアップ (セッションの停止とコード インタープリターの削除)

## 前提条件
- Bedrock AgentCore コード インタープリターにアクセスできる AWS アカウント
- コード インタープリター リソースを作成および管理するための必要な IAM 権限
- S3 操作を実行するための必要な IAM 権限
- 必要な Python パッケージがインストールされている (boto3 と bedrock-agentcore を含む)

## IAM 実行ロールには、以下の IAM ポリシーが添付されている必要があります

~~~ {
    "Version": "2012-10-17",
    "Statement": [
        {
            "Effect": "Allow",
            "Action": [
                "bedrock-agentcore:CreateCodeInterpreter",
                "bedrock-agentcore:StartCodeInterpreterSession",
                "bedrock-agentcore:InvokeCodeInterpreter",
                "bedrock-agentcore:StopCodeInterpreterSession",
                "bedrock-agentcore:DeleteCodeInterpreter",
                "bedrock-agentcore:ListCodeInterpreters",
                "bedrock-agentcore:GetCodeInterpreter"
            ],
            "Resource": "*"
        },
        {
            "Effect": "Allow",
            "Action": [
                "logs:CreateLogGroup",
                "logs:CreateLogStream",
                "logs:PutLogEvents"
            ],
            "Resource": "arn:aws:logs:*:*:log-group:/aws/bedrock-agentcore/code-interpreter*"
        }
    ]
}

日本語訳:
~~~ {
    "Version": "2012-10-17",
    "Statement": [
        {
            "Effect": "許可",
            "Action": [
                "bedrock-agentcore:CreateCodeInterpreter",
                "bedrock-agentcore:StartCodeInterpreterSession",
                "bedrock-agentcore:InvokeCodeInterpreter",
                "bedrock-agentcore:StopCodeInterpreterSession",
                "bedrock-agentcore:DeleteCodeInterpreter",
                "bedrock-agentcore:ListCodeInterpreters",
                "bedrock-agentcore:GetCodeInterpreter"
            ],
            "Resource": "*"
        },
        {
            "Effect": "許可",
            "Action": [
                "logs:CreateLogGroup",
                "logs:CreateLogStream",
                "logs:PutLogEvents"
            ],
            "Resource": "arn:aws:logs:*:*:log-group:/aws/bedrock-agentcore/code-interpreter*"
        }
    ]
}

#### IAM実行ロールには、以下の信頼ポリシーも付与する必要があります。 bedrock-agentcore.amazonaws.com を信頼ポリシーに含めることで、Bedrock Agent サービス自体がこの IAM ロールを引き受け、あなたに代わって操作 (Amazon S3 など) を実行できるようになります。

```{
    "Version": "2012-10-17",
    "Statement": [
        {
            "Effect": "許可",
            "Principal": {
                "AWS": "arn:aws:iam::558228106740:root",
                "Service": [
                    "bedrock-agentcore.amazonaws.com"
                ]
            },
            "Action": "sts:AssumeRole",
            "Condition": {}
        }
    ]
}
```

日本語訳:
バージョンは 2012-10-17 です。ステートメントには以下の内容が含まれています。

効果は「許可」です。
プリンシパルには、AWS アカウント ID 558228106740 のルートユーザーと、"bedrock-agentcore.amazonaws.com" サービスが含まれています。
アクションは "sts:AssumeRole" です。
条件はありません。

さらに、このチュートリアルで説明されている s3 操作を実行するには、AmazonS3FullAccess IAM ポリシーを IAM 実行ロールに付与する必要があります。

## 動作の仕組み

コード実行サンドボックスは、コード インタプリター、シェル、ファイル システムを備えた分離された環境を作成することで、エージェントがユーザーのクエリを安全に処理できるようにします。大規模言語モデルがツールの選択を支援した後、コードがこのセッション内で実行され、その結果がユーザーまたはエージェントに合成のために返されます。

![architecture local](code-interpreter.png)

## 1. 環境のセットアップ

まず、必要なライブラリを import しましょう

In [ ]:
!pip install --upgrade -r requirements.txt

In [ ]:
import json
import boto3
from bedrock_agentcore.tools.code_interpreter_client import code_session, CodeInterpreter
from bedrock_agentcore._utils import endpoints
import time
from typing import Dict, Any, List

## 2. 設定変数

コード インタープリターと S3 操作に必要な設定変数を設定します。また、コード インタープリターに渡す IAM 実行ロールを提供しています。これにより、コード インタープリターは、このロールを引き受けて他の AWS リソースにアクセスできます。このロールには、上記のように S3 の許可が必要です。

In [ ]:
# 設定のセットアップ

日本語訳:

# 設定のセットアップ

次の手順に従って、アプリケーションの設定を行ってください。

1. config.py ファイルを開き、以下の変数を設定します。

 API_KEY = "your_api_key_here"
 SECRET_KEY = "your_secret_key_here"

2. データベースの接続設定を行います。settings.py ファイルを開き、以下の行を編集してください。

 DATABASES = {
     'default': {
         'ENGINE': 'django.db.backends.postgresql',
         'NAME': 'your_db_name',
         'USER': 'your_db_user',
         'PASSWORD': 'your_db_password',
         'HOST': 'localhost',
         'PORT': '5432',
     }
 }

3. 次のコマンドを実行して、マイグレーションを適用します。

 $ python manage.py migrate

4. 開発用サーバーを起動するには、次のコマンドを実行します。

 $ python manage.py runserver

アプリケーションは now http://127.0.0.1:8000/ で実行されています。
execution_role_arn = "<execution-role-arn>"
unique_bucket_name = f"amzn-bucket-{int(time.time())}"
s3_path = f"s3://{unique_bucket_name}"

region = "us-west-2"

#local file that we will upload to Code interpreter and then to an S3 bucket

日本語訳:
# Code interpreter に アップロードし、その後 S3 バケットに アップロードするローカルファイル
local_file = "samples/stats.py"

## 3. エンドポイントの設定

boto3 クライアントを作成するために、データプレーンとコントロールプレーンの両方のエンドポイントを構成する必要があります。

In [ ]:
# Configure endpoints
data_plane_endpoint = endpoints.get_data_plane_endpoint(region)
control_plane_endpoint = endpoints.get_control_plane_endpoint(region)

## 4. AWS クライアントの作成

コントロールプレーンとデータプレーンの両方の操作用に、boto3 クライアントを初期化します。

In [ ]:
# Create boto3 clients

日本語訳:
boto3 クライアントを作成します。

import boto3

ec2 = boto3.client('ec2')
s3 = boto3.resource('s3')

# List EC2 instances
instances = ec2.describe_instances()
for reservation in instances['Reservations']:
    for instance in reservation['Instances']:
        instance_id = instance['InstanceId']
        instance_type = instance['InstanceType']
        print(f'Instance ID: {instance_id}, Type: {instance_type}')

# List S3 buckets
for bucket in s3.buckets.all():
    print(f'Bucket Name: {bucket.name}')
cp_client = boto3.client("bedrock-agentcore-control", 
                        region_name=region,
                        endpoint_url=control_plane_endpoint)

dp_client = boto3.client("bedrock-agentcore", 
                        region_name=region,
                        endpoint_url=data_plane_endpoint)

## 5. コードインタープリターの作成

特定の構成パラメーターを使用してコードインタープリターインスタンスを作成します。

コードインタープリターを構成する際、ネットワーク設定 (Sandbox、Public、または VPC)、依存関係の構成、セキュリティ設定を選択でき、また、IAM ランタイムロールを通じてコードインタープリターがアクセスできる AWS リソースを定義するアクセス許可を設定できます。

In [ ]:
# コードインタープリターを作成

日本語訳:

# コードインタープリターを作成

import sys

def eval_expr(expr):
    """
    eval_expr(expr) は、与えられた expr を評価し、その結果を返します。
    expr は、数値、変数、演算子 (+、-、*、/)、および丸括弧で構成される文字列です。
    """
    # ここにコードを記述します。

def test_eval_expr():
    """
    eval_expr の単体テストを行います。
    """
    # ここにテストコードを記述します。
    print('すべてのテストが完了しました。')

if __name__ == '__main__':
    test_eval_expr()
unique_name = f"s3InteractionEnv_{int(time.time())}"
interpreter_response = cp_client.create_code_interpreter(
    name=unique_name,
    description="Environment for S3 file operations",
    executionRoleArn=execution_role_arn,
    networkConfiguration={
        'networkMode': 'PUBLIC'
    }
)
interpreter_id = interpreter_response["codeInterpreterId"]
print(f"Created interpreter: {interpreter_id}")

## 6. セッションの開始

コード インタープリター内でセッションを作成し、コードを実行します。

日本語訳:
## 6. セッションの開始

コード インタープリターの中で セッションを作成し、コードを実行します。

In [ ]:
# Start session

セッションを開始します。

以下のテキストを英語から日本語に翻訳します。半角英数字の前後には半角スペースを挿入し、コード、コマンド、変数名、関数名などの技術的な用語はそのまま残します。
session_response = dp_client.start_code_interpreter_session(
    codeInterpreterIdentifier=interpreter_id,
    name="s3InteractionSession",
    sessionTimeoutSeconds=900
)
session_id = session_response["sessionId"]
print(f"Created session: {session_id}")

## 7. ツール実行のためのヘルパー関数

コードインタープリターツールの呼び出しを簡素化するためのユーティリティ関数を定義します。

In [ ]:
def call_tool(tool_name: str, arguments: Dict[str, Any]) -> Dict[str, Any]:
    response = dp_client.invoke_code_interpreter(
        codeInterpreterIdentifier=interpreter_id,
        sessionId=session_id,
        name=tool_name,
        arguments=arguments
    )
    for event in response["stream"]:
        return json.dumps(event["result"], indent=2)

## 8. テストコードの実行

### 8.1 簡単な Hello World の例でコード インタープリターをテストします。

日本語訳:
## 8. コードの実行をテストする

### 8.1 簡単な Hello World の例で、コード インタプリタをテストします。

In [ ]:
# Test code execution

テストコードの実行

以下のように、 `run_test()` 関数を呼び出すことで、テストコードを実行できます。

```python
if __name__ == "__main__":
    run_test()
```

この関数は、 `tests/` ディレクトリ内のすべての `.py` ファイルを検索し、それらのファイル内の `TestCase` クラスを実行します。テストの結果は、標準出力に出力されます。

テストが失敗した場合は、 `pytest` を使用して、より詳細な情報を取得することができます。 `pytest` は、 Python の単体テストフレームワークです。以下のコマンドを実行すると、 `pytest` がインストールされます。

```
pip install pytest
```

その後、以下のコマンドを実行すると、 `tests/` ディレクトリ内のすべてのテストが実行されます。

```
pytest tests/
```
# S3 操作

日本語訳:

Amazon S3 は、データを任意の場所から安全に保存および取得できるようにするオブジェクトストレージサービスです。S3 は、99.999999999% の耐久性を備えており、データを安全に保存できます。

S3 では、バケットを使用してデータを格納します。バケットは、オブジェクトの格納と管理のためのコンテナです。各オブジェクトは、バケット内に格納されます。

S3 の主な操作は次のとおりです。

1. バケットの作成: `aws s3 mb s3://bucket-name`
2. オブジェクトのアップロード: `aws s3 cp file.txt s3://bucket-name`
3. オブジェクトのダウンロード: `aws s3 cp s3://bucket-name/file.txt file.txt`
4. オブジェクトの一覧表示: `aws s3 ls s3://bucket-name`
5. バケットの削除: `aws s3 rb s3://bucket-name`

S3 は、データの耐久性、可用性、セキュリティを確保するための多くの機能を提供しています。アクセス制御リスト (ACL) やバケットポリシーを使用して、オブジェクトへのアクセスを制御できます。また、バージョニングを有効にすると、オブジェクトの以前のバージョンを保持できます。さらに、ライフサイクルポリシーを設定して、オブジェクトの有効期限を管理できます。
print("executing shell command \n")
command_response = call_tool("executeCommand",
                              {"command": "echo 'Hello World'"})
print(f"command result: {command_response}")

# Parse and display results

# 結果を解析して表示する

results = parse_data( raw_data )
for result in results:
    print( f"Name: { result['name'] }" )
    print( f"Score: { result['score'] }" )
    print( )  # 空行を挿入

# 結果を JSON として保存
output_json = json.dumps( results )
with open( "results.json", "w" ) as f:
    f.write( output_json )

print( "Done!" )
command_results = json.loads(command_response)
print(command_results['structuredContent']['stdout'])

### 8.2 次に、boto3 を PIP を使ってサンドボックスにインストールしましょう

日本語訳:
8.2 次に、サンドボックス内で PIP を使って boto3 をインストールしましょう。

In [ ]:
# Test code execution

テストコードの実行

以下のように、 `run_test()` 関数を呼び出すことで、テストコードを実行できます。

```python
if __name__ == "__main__":
    run_test()
```

この関数は、 `tests/` ディレクトリ内のすべての `.py` ファイルを検索し、それらのファイル内の `TestCase` クラスを実行します。テストの結果は、標準出力に出力されます。

テストが失敗した場合は、 `pytest` を使用してデバッグすることをお勧めします。 `pytest` は、より詳細な出力とデバッグ機能を提供してくれます。

```bash
pip install pytest
pytest tests/
```
# S3 操作

日本語訳:

Amazon S3 は、データを任意の場所から安全に保存および取得できるようにするオブジェクトストレージサービスです。S3 を使用すると、いつでもウェブサービスからデータを取得できます。

S3 では、データはバケットに格納されます。バケットは、オブジェクトを格納するための S3 のコンテナです。オブジェクトは、ファイルとそのファイルに関連付けられたメタデータの集合です。

S3 では、次の操作を実行できます。

- 新しいバケットを作成する: `create_bucket` 関数を使用します。
- オブジェクトをアップロードする: `put_object` 関数を使用します。
- オブジェクトをダウンロードする: `get_object` 関数を使用します。
- オブジェクトを削除する: `delete_object` 関数を使用します。
- バケットを削除する: `delete_bucket` 関数を使用します。

これらの操作の詳細については、AWS のドキュメントを参照してください。
print("executing shell command \n")
command_response = call_tool("executeCommand",
                              {"command": "pip install boto3"})

# Parse and display results

# 結果を解析して表示する

results = parse_data(data_file)  # data_file からデータを解析

print("Total records: ", len(results))  # 総レコード数を出力

for result in results:
    print(f"Name: {result['name']}, Score: {result['score']}")  # 各レコードの name と score を出力
command_results = json.loads(command_response)
print(command_results['structuredContent']['stdout'])

## 9. ファイル操作とS3との対話によるコマンド実行

#### 9.1 ローカルファイルをサンドボックスに書き込む

日本語訳:
あなたは翻訳の専門家です。以下のテキストを英語から日本語に翻訳してください。
半角英数字の前後には半角スペースを挿入する。コードやコマンド、変数名、関数名などの技術的な用語は翻訳せず、そのまま残してください。

翻訳するテキスト:
```
!echo "hello world" > hello.txt
```

In [ ]:
# Write file to sandbox

sandbox に ファイルを書き込む

import os

def write_file(filename, contents):
    """
    filename (str): 書き込むファイルの名前
    contents (str): ファイルに書き込む内容
    """
    with open(os.path.join('/sandbox', filename), 'w') as f:
        f.write(contents)

# 使用例
write_file('hello.txt', 'Hello, World!')
print("Writing file to sandbox")
try:
    with open(local_file, 'r', encoding='utf-8') as local_file_content:
        local_file_content = local_file_content.read()
except FileNotFoundError:
    print(f"Error: The file '{local_file}' was not found.")
except Exception as e:
    print(f"An error occurred: {e}")

files_to_create = [{
        "path": "stats.py",
        "text": local_file_content
}]
write_files_response = call_tool("writeFiles", {"content": files_to_create})
print(f"write files result: {write_files_response}")

#### 9.2 コード インタープリターを使用して S3 バケットを作成する

日本語訳:

この項では、AWS SDK を使用して S3 バケットを作成する方法を説明します。以下の手順に従ってください。

1. まず、 `aws_s3` リソースを使用して新しい S3 バケットを作成します。

```python
resource = boto3.resource('s3')
bucket = resource.Bucket('my-bucket-name')
bucket.create(
    CreateBucketConfiguration={
        'LocationConstraint': 'ap-northeast-1'
    }
)
```

2. バケットが正常に作成されたことを確認するには、 `bucket.name` と `bucket.creation_date` を出力します。

```python
print(bucket.name)
print(bucket.creation_date)
```

3. オプションで、バケットにオブジェクトをアップロードすることもできます。 `bucket.upload_file()` メソッドを使用して、ローカル ファイルをバケットにアップロードします。

```python
bucket.upload_file('/path/to/local/file', 'object-name')
```

4. バケットの内容を一覧表示するには、 `bucket.objects.all()` を使用します。

```python
for obj in bucket.objects.all():
    print(obj.key)
```

これで、Python コードを使用して S3 バケットを作成し、オブジェクトをアップロードおよび一覧表示する方法がわかりました。

In [ ]:
# S3 Operations

日本語訳:

Amazon S3 は、データを任意の場所から安全に保存および取得できるオブジェクトストレージサービスです。S3 では、データを 1 つ以上の バケット に格納します。バケットは、データを格納するための コンテナ のようなものです。

S3 では、以下の操作を実行できます。

- バケットの作成
- データのアップロード
- アクセス許可の設定
- データの取得
- データの削除

S3 には、データを保存するための複数の ストレージクラス があります。ストレージクラスは、データのアクセス頻度に基づいて選択できます。頻繁にアクセスするデータには、高速で低コストの S3 Standard を使用します。頻繁にアクセスしないデータには、低コストの S3 Glacier を使用します。

S3 では、データの転送料金、ストレージ料金、その他の料金が発生します。料金は、使用するリージョン、ストレージクラス、データ転送量などによって異なります。

S3 は、高い可用性、耐久性、セキュリティを備えています。データは、複数の施設に冗長的に保存されます。また、データの暗号化、アクセス制御、監査ログなどの機能により、データの保護が可能です。
print("\nCreating S3 bucket")
create_s3_response = call_tool("executeCommand",
                              {"command": f"aws s3 mb {s3_path} --region {region}"})
print(f"create result: {create_s3_response}")

#### 9.3 Upload file from code interpreter by running command to the S3 Bucket (created above)

日本語訳:

#### 9.3 コード インタープリターから実行コマンドによって、上記で作成した S3 バケットにファイルをアップロードする

In [ ]:
print("\nUploading file to S3")
upload_to_s3_response = call_tool("executeCommand",
                                 {"command": f"aws s3 cp {files_to_create[0]['path']} {s3_path}"})
print(f"upload result: {upload_to_s3_response}")



#### 9.4 コード インタープリターでコマンドを実行して S3 バケットからファイルを一覧表示する

日本語訳:

コード インタープリターでコマンドを実行することにより、S3 バケットからファイルを一覧表示できます。以下の `aws s3 ls` コマンドを使用して、バケット内のファイルとプレフィックス (ディレクトリのようなもの) を一覧表示します。

```
aws s3 ls s3://{{ bucket }}/ --recursive --human-readable --summarize
```

このコマンドは、指定された S3 バケット内のすべてのファイルとプレフィックスを再帰的に一覧表示します。`--human-readable` フラグにより、ファイルサイズが読みやすい形式で表示されます。`--summarize` フラグを使用すると、バケットの合計ファイル数とサイズの概要が表示されます。

In [ ]:
print("\nListing files in S3")
list_s3_response = call_tool("executeCommand",
                            {"command": f"aws s3 ls {s3_path}"})
print(f"list result: {list_s3_response}")

## 10. リソースのクリーンアップ

セッションを停止し、インタープリターを削除することでリソースをクリーンアップします。

In [ ]:
# Cleanup

日本語訳:

この手順は、不要なファイルやディレクトリを削除して、システムをクリーンアップするためのものです。

1. 一時ファイルを削除するには、次のコマンドを実行します: `rm -rf /tmp/*`

2. キャッシュをクリアするには、次のコマンドを実行します: `sudo apt-get clean`

3. 古いカーネルイメージを削除するには、次のコマンドを実行します: `sudo apt-get autoremove --purge`

4. 不要なパッケージを削除するには、次のコマンドを実行します: `sudo apt-get autoremove`

5. 最後に、ディスクの空き領域を確保するために、次のコマンドを実行します: `sudo apt-get autoclean`

システムがクリーンアップされ、ディスク領域が解放されました。
print("Cleaning up session and interpreter")
dp_client.stop_code_interpreter_session(
    codeInterpreterIdentifier=interpreter_id,
    sessionId=session_id
)
print("Session stopped successfully")

cp_client.delete_code_interpreter(codeInterpreterId=interpreter_id)
print("Interpreter deleted successfully")